In [ ]:
!pip install librosa

In [ ]:
!pip install numpy


In [ ]:
!pip install --upgrade pip


In [ ]:
SR =  16000
window_size = 1024
hop_length = 256

patch_size = 128 # roughly 33 seconds


# Preprocess stage
Load audio set and store into Spectrogram

In [ ]:
import librosa
from librosa.util import find_files
from librosa import load

import os, re
import numpy as np


def SaveSpectrogram(y_mix,y_vocal, filename, orig_sr=44100, target_sr=SR) :
    y_mix = librosa.resample(y_mix,orig_sr=orig_sr,target_sr=target_sr)
    y_vocal = librosa.resample(y_vocal,orig_sr=orig_sr,target_sr=target_sr)


    S_mixt = np.abs(librosa.stft(y_mix,n_fft=window_size,hop_length=hop_length)).astype(np.float32)

    S_vocalt = np.abs(librosa.stft(y_vocal,n_fft=window_size,hop_length=hop_length)).astype(np.float32)

    S_mix = librosa.power_to_db(S_mixt**2)

    S_vocal = librosa.power_to_db(S_vocalt**2)

    norm = S_mix.max()
    S_mix /= norm

    S_vocal /= norm

    #np.savez(os.path.join('./Spectrogram3',filename+'.npz'),mix=S_mix,inst=S_inst ,vocal=S_vocal)
    np.savez(os.path.join('./SpectrogramTEST',filename+'.npz'),mix=S_mix,vocal=S_vocal)



In [ ]:
import librosa
from librosa.util import find_files
from librosa import load
import os


In [ ]:


# Save Spectrogram
def Mixer() :
    '''
    source_0: original wav file

    source_1 : vocal wav file
    '''
    #Audiolist = os.listdir('./data')
    Audiolist = os.listdir('./test')
    for audio in Audiolist :
        try :
  #          audio_path = os.path.join('./data/'+audio)
            audio_path = os.path.join('./test/'+audio)
            print("Song : %s" % audio)
  #          if os.path.exists(os.path.join('./Spectrogram3',audio+'.npz')) :
            if os.path.exists(os.path.join('./SpectrogramTEST',audio+'.npz')) :
                print("Already exist!! Skip....")
                continue
            aud = find_files(audio_path,ext="wav")

            mix,_ = load(aud[0], sr=None)

            vocal,_ = load(aud[1], sr=None)
            print("Saving...")

            SaveSpectrogram(mix, vocal, audio)
        except IndexError as e :
            print("Wrong Directory")
            pass



In [ ]:
    Mixer()
    print("Complete!!!!")

Song : Al James - Schoolboy Facination
Saving...
Song : AM Contra - Heart Peripheral
Saving...
Song : Angels In Amplifiers - I'm Alright
Saving...
Song : Arise - Run Run Run
Saving...
Song : Ben Carrigan - We'll Talk About It All Tonight
Saving...
Song : BKS - Bulldozer
Saving...
Song : BKS - Too Much
Saving...
Song : Bobby Nobody - Stitch Up
Saving...
Song : Buitraker - Revo X
Saving...
Song : Carlos Gonzalez - A Place For Us
Saving...
Song : Cristina Vane - So Easy
Saving...
Song : Detsky Sad - Walkie Talkie
Saving...
Song : Enda Reilly - Cur An Long Ag Seol
Saving...
Song : Forkupines - Semantics
Saving...
Song : Georgia Wonder - Siren
Saving...
Song : Girls Under Glass - We Feel Alright
Saving...
Song : Hollow Ground - Ill Fate
Saving...
Song : James Elder & Mark M Thompson - The English Actor
Saving...
Song : Juliet's Rescue - Heartbeats
Saving...
Song : Little Chicago's Finest - My Own
Saving...
Song : Louis Cressy Band - Good Time
Saving...
Song : Signe Jakobsen - What Have You 

# Testing the model


In [ ]:
import tensorflow as tf
from tensorflow import keras
import librosa
import numpy as np
import soundfile
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras import backend as K
from librosa.util import find_files
from numpy import load


from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, Dropout, LeakyReLU, Conv2DTranspose as Deconv2D, Activation, Concatenate, BatchNormalization
from tensorflow.keras.layers import UpSampling2D



from tensorflow import keras
import librosa



SAMPLE_RATE=16000
WINDOW_SIZE=1024
HOP_LENGTH=256

PATCH_SIZE =128 #256
SAMPLE_STRIDE =64
BATCH_SIZE = 32
EPOCH = 20




trained_model = keras.models.load_model('/content/Model_0.5B16.keras')



def LoadAudio(file_path) :
    WINDOW_SIZE=1024
    HOP_LENGTH=256
    SR =  16000

    y, sr = librosa.load(file_path,sr=SR)

    stft = librosa.stft(y,n_fft=WINDOW_SIZE,hop_length=HOP_LENGTH)

    mag, phase = librosa.magphase(stft)
    mag = mag**2
    mag = librosa.power_to_db(mag)


    mag = mag.astype(np.float32)


    norm = mag.max()
    mag /= norm








    return mag, phase, norm

test_song_mag, test_song_phase, norm = LoadAudio('./mixture.wav')


x_test = []
num_chunks = test_song_mag.shape[1]//128
for i in range(num_chunks):
    chunk = test_song_mag[:512, i * 128 : (i + 1) * 128, np.newaxis]
    x_test.append(chunk)

x_test = np.asarray(x_test, dtype = np.float32)
y_test = trained_model.predict(x_test)
print("max prediction")
print(y_test.max())


reconstructed_song =  np.zeros((512, 128*len(y_test), 1))
for i, chunk in enumerate(y_test):
    reconstructed_song[:, i * 128:(i + 1) * 128, :] = chunk



reconstructed_song = np.squeeze(reconstructed_song, axis=-1)


reconstructed_song = reconstructed_song*norm


reconstructed_song =  reconstructed_song.astype(np.float32)
reconstructed_song = librosa.db_to_power(reconstructed_song)
reconstructed_song = reconstructed_song ** 0.5



test_song_phase = test_song_phase[:512,:128*len(x_test)]
test_song_angle = np.angle(test_song_phase)

spectrum = reconstructed_song * np.exp(1j*test_song_phase)



final_audio = librosa.istft(spectrum)



SR =  16000

soundfile.write("./output_song_db.wav", final_audio, SR)


OSError: No file or directory found at ./best_model_db.keras